In [1]:
import numpy as np
from sklearn.datasets import load_boston

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

In [2]:
dataset = load_boston()

In [3]:
X = dataset['data']
y = dataset['target']

In [4]:
X_train , X_test, y_train, y_test = train_test_split(X,y)

In [5]:
X_train_A, X_train_B, y_train_A, y_train_B = train_test_split(X_train, y_train)

In [7]:
np.logspace(-2, 2);

In [8]:
gs_ridge = RidgeCV(alphas=np.logspace(-2,2))

In [9]:
gs_ridge.fit(X_train_A, y_train_A)

RidgeCV(alphas=array([  1.00000e-02,   1.20679e-02,   1.45635e-02,   1.75751e-02,
         2.12095e-02,   2.55955e-02,   3.08884e-02,   3.72759e-02,
         4.49843e-02,   5.42868e-02,   6.55129e-02,   7.90604e-02,
         9.54095e-02,   1.15140e-01,   1.38950e-01,   1.67683e-01,
         2.02359e-01,   2...     3.90694e+01,   4.71487e+01,   5.68987e+01,   6.86649e+01,
         8.28643e+01,   1.00000e+02]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [12]:
gs_ridge.alpha_

0.1151395399326447

In [13]:
meta1 = gs_ridge.predict(X_train_B)

In [14]:
meta1.shape

(95,)

In [16]:
gs_knn = GridSearchCV(KNeighborsRegressor(), param_grid={'n_neighbors' : np.arange(1, 20)})
gs_knn.fit(X_train_A, y_train_A)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [17]:
gs_knn.best_params_

{'n_neighbors': 3}

In [18]:
meta2 = gs_knn.predict(X_train_B)

In [19]:
gs_RF = GridSearchCV(RandomForestRegressor(n_estimators=30, n_jobs=-1),param_grid={'max_depth' : np.arange(2,15)})
gs_RF.fit(X_train_A, y_train_A)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [20]:
gs_RF.best_params_

{'max_depth': 10}

In [21]:
meta3 = gs_RF.predict(X_train_B)

In [22]:
from sklearn.metrics import mean_absolute_error

In [23]:
print("KNN: ", mean_absolute_error(y_test,gs_knn.predict(X_test)))
print("Ridge: ", mean_absolute_error(y_test,gs_ridge.predict(X_test)))
print("RF: ", mean_absolute_error(y_test,gs_RF.predict(X_test)))

KNN:  4.32965879265
Ridge:  3.53035819388
RF:  2.6006913911


In [25]:
new_X_train = np.vstack((meta1, meta2, meta3)).T

In [26]:
meta_ridge = RidgeCV(np.logspace(-2,2))


In [27]:
meta_ridge.fit(new_X_train, y_train_B)

RidgeCV(alphas=array([  1.00000e-02,   1.20679e-02,   1.45635e-02,   1.75751e-02,
         2.12095e-02,   2.55955e-02,   3.08884e-02,   3.72759e-02,
         4.49843e-02,   5.42868e-02,   6.55129e-02,   7.90604e-02,
         9.54095e-02,   1.15140e-01,   1.38950e-01,   1.67683e-01,
         2.02359e-01,   2...     3.90694e+01,   4.71487e+01,   5.68987e+01,   6.86649e+01,
         8.28643e+01,   1.00000e+02]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [28]:
new_X_test = np.vstack((gs_ridge.predict(X_test), gs_knn.predict(X_test),gs_RF.predict(X_test))).T

In [29]:
y_pred = meta_ridge.predict(new_X_test)

In [30]:
mean_absolute_error(y_test,y_pred)

2.5973258791181908

In [31]:
from sklearn.preprocessing import PolynomialFeatures

In [32]:
pf = PolynomialFeatures(include_bias=False)
pf.fit(new_X_train)
pf_new_X_train = pf.transform(new_X_train)
pf_new_X_test = pf.transform(new_X_test)

In [33]:
meta_ridge = RidgeCV(np.logspace(-2,2))
meta_ridge.fit(pf_new_X_train, y_train_B)
y_pred = meta_ridge.predict(pf_new_X_test)
mean_absolute_error(y_test,y_pred)

2.3930197918450014